# verifying the existence of an email address using python
# 

In [ ]:
#improting blibrarys

In [ ]:
import re   ##regular expression matching
import smtplib   ## sening email
import dns.resolver ##DNS lookups

In [ ]:


# Regular expression for email syntax validation
email_regex = r'^[\w-]+(\.[\w-]+)*@(\w+\.)+\w{2,}$'

# Email address to verify
input_address = input('Please enter the email address to verify: ')
address_to_verify = str(input_address)

# Syntax check
match = re.match(email_regex, address_to_verify)
if match is None:
    print('Invalid Syntax')
    raise ValueError('Invalid Syntax')

# Splitting email address into username and domain
username, domain = address_to_verify.split('@')

# Checking if there is a subdomain
subdomains = domain.split('.')
if len(subdomains) > 3:
    print('Invalid email address - too many subdomains')
elif len(subdomains) == 3:
    print('Valid email address with subdomain')
else:
    print('Valid email address')

# Perform DNS MX record lookup for domain
try:
    mx_records = dns.resolver.query(domain, 'MX')
except dns.exception.DNSException:
    print('Error performing MX record lookup')
    exit()

# Get the highest priority MX record (the one with lowest preference value)
mx_record = sorted(mx_records, key=lambda x: x.preference)[0].exchange.to_text()

# SMTP conversation to verify email address
try:
    with smtplib.SMTP() as smtp:
        smtp.set_debuglevel(0)
        smtp.connect(mx_record)
        smtp.helo(smtp.local_hostname)
        smtp.mail('test@example.com')
        code, message = smtp.rcpt(address_to_verify)
except smtplib.SMTPException:
    print('Error sending email')
    exit()

# Check SMTP response code to determine email verification result
if code == 250:
    print('Success')
else:
    print('Bad')
